In [1]:
import torch
import sae_lens

/mnt/c/Users/huohu/Documents/code/SAEGeometry/config/saegeometry-1tp4usyN-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
layers = 6
device = "cuda" if torch.cuda.is_available() else "cpu"
sae_list = []
release = "pythia-70m-deduped-res-sm"
sae_id = f"blocks.0.hook_resid_pre"
model_name = "pythia-70m-deduped"
sae_list.append(
    sae_lens.SAE.from_pretrained(
        release=release, sae_id=sae_id, device=device
    )[0]
)
for layer in range(layers):
    sae_id = f"blocks.{layer}.hook_resid_post"
    sae_list.append(
        sae_lens.SAE.from_pretrained(
            release=release, sae_id=sae_id, device=device
        )[0]
    )

model = sae_lens.HookedSAETransformer.from_pretrained(model_name).to(device)

The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


Loaded pretrained model pythia-70m-deduped into HookedTransformer
Moving model to device:  cuda


In [3]:
import datasets
dataset = datasets.load_dataset("Salesforce/wikitext", "wikitext-2-raw-v1")["train"]

In [4]:
from tqdm import tqdm
save_name="wiki_pythia_res_post.pt"
length = int(0.1*len(dataset))
doc_len = 0
freq_mean_global = 0
layers = 6
freqs = torch.zeros(layers + 1, sae_list[0].cfg.d_sae).to(device)

for idx in tqdm(range(length)):
    # loop begin, fuck indent
    example = dataset[idx]
    tokens = model.to_tokens([example["text"]], prepend_bos=True)
    _, cache = model.run_with_cache_with_saes(tokens, saes=sae_list)
    local_doc_len = cache["blocks.0.hook_resid_post.hook_sae_acts_post"].shape[1]
    freq = torch.zeros_like(freqs)
    for layer in range(layers):
        prompt = f"blocks.{layer}.hook_resid_pre.hook_sae_acts_post"
        prompt2 = f"blocks.{layer}.hook_resid_post.hook_sae_acts_post"
        if layer == 0:
            freq[layer] = (cache[prompt] > 1e-3)[0].sum(0) / local_doc_len

        freq[layer + 1] = (cache[prompt2] > 1e-3)[0].sum(0) / local_doc_len
    new_doc_len = doc_len + local_doc_len
    if idx == 0:
        freq_mean_global = freq
    else:
        freq_mean_global = (
            freq_mean_global * doc_len / new_doc_len
            + freq * local_doc_len / new_doc_len
        )
    doc_len = new_doc_len
torch.save(freq_mean_global, save_name)

100%|██████████| 3671/3671 [01:14<00:00, 49.16it/s]


In [7]:
math_dataset = datasets.load_dataset("hbin0701/abstract_math")["train"]["input"]

In [8]:
from tqdm import tqdm
save_name="math_pythia_res_post.pt"
length = int(0.01*len(math_dataset))
doc_len = 0
freq_mean_global = 0
layers = 6
freqs = torch.zeros(layers + 1, sae_list[0].cfg.d_sae).to(device)

for idx in tqdm(range(length)):
    # loop begin, fuck indent
    example = math_dataset[idx]
    tokens = model.to_tokens([example], prepend_bos=True)
    _, cache = model.run_with_cache_with_saes(tokens, saes=sae_list)
    local_doc_len = cache["blocks.0.hook_resid_post.hook_sae_acts_post"].shape[1]
    freq = torch.zeros_like(freqs)
    for layer in range(layers):
        prompt = f"blocks.{layer}.hook_resid_pre.hook_sae_acts_post"
        prompt2 = f"blocks.{layer}.hook_resid_post.hook_sae_acts_post"
        if layer == 0:
            freq[layer] = (cache[prompt] > 1e-3)[0].sum(0) / local_doc_len

        freq[layer + 1] = (cache[prompt2] > 1e-3)[0].sum(0) / local_doc_len
    new_doc_len = doc_len + local_doc_len
    if idx == 0:
        freq_mean_global = freq
    else:
        freq_mean_global = (
            freq_mean_global * doc_len / new_doc_len
            + freq * local_doc_len / new_doc_len
        )
    doc_len = new_doc_len
torch.save(freq_mean_global, save_name)

100%|██████████| 10000/10000 [03:23<00:00, 49.20it/s]


In [9]:
freq_mean_global.max()

tensor(0.9884, device='cuda:0')

In [10]:
import datasets
ds = datasets.load_dataset("iamtarun/code_contest_python3_alpaca")['train']
from tqdm import tqdm
save_name="code_pythia_res_post.pt"
length = int(len(ds))
doc_len = 0
freq_mean_global = 0
layers = 6
freqs = torch.zeros(layers + 1, sae_list[0].cfg.d_sae).to(device)

for idx in tqdm(range(length)):
    # loop begin, fuck indent
    example = ds[idx]
    tokens = model.to_tokens([example["code"]], prepend_bos=True)
    _, cache = model.run_with_cache_with_saes(tokens, saes=sae_list)
    local_doc_len = cache["blocks.0.hook_resid_post.hook_sae_acts_post"].shape[1]
    freq = torch.zeros_like(freqs)
    for layer in range(layers):
        prompt = f"blocks.{layer}.hook_resid_pre.hook_sae_acts_post"
        prompt2 = f"blocks.{layer}.hook_resid_post.hook_sae_acts_post"
        if layer == 0:
            freq[layer] = (cache[prompt] > 1e-3)[0].sum(0) / local_doc_len

        freq[layer + 1] = (cache[prompt2] > 1e-3)[0].sum(0) / local_doc_len
    new_doc_len = doc_len + local_doc_len
    if idx == 0:
        freq_mean_global = freq
    else:
        freq_mean_global = (
            freq_mean_global * doc_len / new_doc_len
            + freq * local_doc_len / new_doc_len
        )
    doc_len = new_doc_len
torch.save(freq_mean_global, save_name)

100%|██████████| 8139/8139 [03:11<00:00, 42.43it/s]
